In [1]:
%pip install pandas numpy scikit-learn matplotlib seaborn statsmodels prophet tabulate

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Core imports
import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error

/Users/sanjaym/Desktop/projects/simple_ml_predictor/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
# Load CSV (do not use index_col so all columns are available)
df = pd.read_csv('data/data.csv')
print('Rows:', len(df))
df.head()

# Prepare hourly time series DataFrame
df_hourly = df.groupby(['date', 'hour'])[['number_of_transactions', 'total_amount']].sum().reset_index()

# construct a datetime 'ds' column
df_hourly['ds'] = pd.to_datetime(df_hourly['date'].astype(str) + ' ' + df_hourly['hour'].astype(str) + ':00:00')

Rows: 744


In [4]:
# 1. Normalize the two impact columns (Min-Max Scaling)
min_tx = df_hourly['number_of_transactions'].min()
max_tx = df_hourly['number_of_transactions'].max()
# Handle case where min == max (to avoid division by zero)
range_tx = max_tx - min_tx
df_hourly['norm_transactions'] = (df_hourly['number_of_transactions'] - min_tx) / range_tx if range_tx != 0 else 0

min_amt = df_hourly['total_amount'].min()
max_amt = df_hourly['total_amount'].max()
range_amt = max_amt - min_amt
df_hourly['norm_amount'] = (df_hourly['total_amount'] - min_amt) / range_amt if range_amt != 0 else 0


# 2. Apply the 60:40 weighting to the normalized values to create the target variable 'y'
# 60% weight on normalized transactions, 40% on normalized amount.
WEIGHT_TX = 0.60
WEIGHT_AMT = 0.40
df_hourly['y'] = (df_hourly['norm_transactions'] * WEIGHT_TX) + (df_hourly['norm_amount'] * WEIGHT_AMT)

# Select the necessary columns for Prophet
df_ts = df_hourly[['ds', 'y']].sort_values('ds').reset_index(drop=True)
print("\nTarget Variable 'y' Head (Weighted Impact Score):")
print(df_ts.head())

# Feature engineering (exogenous regressors)
df_features = df_ts.copy()


Target Variable 'y' Head (Weighted Impact Score):
                   ds         y
0 2024-01-01 00:00:00  0.033140
1 2024-01-01 01:00:00  0.008085
2 2024-01-01 02:00:00  0.017125
3 2024-01-01 03:00:00  0.024624
4 2024-01-01 04:00:00  0.008839


In [5]:
# A. Cyclical and Binary Features
df_features['hour'] = df_features['ds'].dt.hour
df_features['day_of_week'] = df_features['ds'].dt.dayofweek # Mon=0, Sun=6
df_features['day_of_month'] = df_features['ds'].dt.day # Regressor for monthly cycles

# Cyclical Encoding for Hour (24-hour cycle)
df_features['hour_sin'] = np.sin(2 * np.pi * df_features['hour'] / 24)
df_features['hour_cos'] = np.cos(2 * np.pi * df_features['hour'] / 24)

# Cyclical Encoding for Day of Week (7-day cycle)
df_features['day_sin'] = np.sin(2 * np.pi * df_features['day_of_week'] / 7)
df_features['day_cos'] = np.cos(2 * np.pi * df_features['day_of_week'] / 7)

# Is it a Weekend?
df_features['is_weekend'] = df_features['day_of_week'].isin([5,6]).astype(int)

# Drop temporary raw columns
df_features = df_features.drop(columns=['hour','day_of_week'])

# Add lag features for daily and weekly seasonality
df_features['lag_24h'] = df_features['y'].shift(24)
df_features['lag_168h'] = df_features['y'].shift(168)

# Drop initial rows with NaNs from lags
df_prepared = df_features.dropna().reset_index(drop=True)
print(f'\nOriginal rows: {len(df_ts)}, After lag & dropna: {len(df_prepared)}')
df_prepared.head()


Original rows: 744, After lag & dropna: 576


,ds,y,day_of_month,hour_sin,hour_cos,day_sin,day_cos,is_weekend,lag_24h,lag_168h
0,2024-01-08 00:00:00,0.033041,8,0.000000,1.000000,0.0,1.0,0,0.025796,0.033140
1,2024-01-08 01:00:00,0.033432,8,0.258819,0.965926,0.0,1.0,0,0.033264,0.008085
2,2024-01-08 02:00:00,0.042954,8,0.500000,0.866025,0.0,1.0,0,0.041221,0.017125
3,2024-01-08 03:00:00,0.048875,8,0.707107,0.707107,0.0,1.0,0,0.008341,0.024624
4,2024-01-08 04:00:00,0.008804,8,0.866025,0.500000,0.0,1.0,0,0.024621,0.008839


In [6]:
# Train / test split: use 7 days (one week) for test since the dataset is one month long
TEST_SIZE = 7 * 24
if TEST_SIZE >= len(df_prepared):
    TEST_SIZE = max(24, len(df_prepared) // 5)  # fallback: 20% or at least 24 hours
df_train = df_prepared.iloc[:-TEST_SIZE].reset_index(drop=True)
df_test = df_prepared.iloc[-TEST_SIZE:].reset_index(drop=True)
print(f'\nTraining set size: {len(df_train)} rows.')
print(f'Testing set size: {len(df_test)} rows ({TEST_SIZE} hours).')

# Fit Prophet with all exogenous regressors, including the new 'day_of_month'
model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=False, interval_width=0.95)
exogenous_features = [
    'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'is_weekend',
    'lag_24h', 'lag_168h', 'day_of_month' # <<< New feature added
]

for feature in exogenous_features:
    model.add_regressor(feature)

# Prophet expects columns: ds, y, and the added regressor columns in the training DataFrame
model.fit(df_train[['ds','y'] + exogenous_features])

# --- Evaluation (using the test set) ---
df_future_test = df_test[['ds'] + exogenous_features].copy()
forecast_test = model.predict(df_future_test)
y_pred = forecast_test['yhat'].values
y_true = df_test['y'].values
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print('\nModel Evaluation on Test Data:')
print(f'Mean Absolute Error (MAE): {mae:,.4f}') # Increased precision since 'y' is now normalized
print(f'Root Mean Squared Error (RMSE): {rmse:,.4f}')

03:06:19 - cmdstanpy - INFO - Chain [1] start processing
03:06:19 - cmdstanpy - INFO - Chain [1] done processing



Training set size: 408 rows.
Testing set size: 168 rows (168 hours).

Model Evaluation on Test Data:
Mean Absolute Error (MAE): 0.0695
Root Mean Squared Error (RMSE): 0.0954


In [7]:
# --- Generate Future Forecast for Next Month (30 days) ---
FUTURE_DAYS = 30
future = model.make_future_dataframe(periods=FUTURE_DAYS * 24, freq='H')

# 1. Populate the exogenous features for the future period
future['hour'] = future['ds'].dt.hour
future['day_of_week'] = future['ds'].dt.dayofweek
future['day_of_month'] = future['ds'].dt.day # New Regressor for monthly cycles

future['hour_sin'] = np.sin(2 * np.pi * future['hour'] / 24)
future['hour_cos'] = np.cos(2 * np.pi * future['hour'] / 24)
future['day_sin'] = np.sin(2 * np.pi * future['day_of_week'] / 7)
future['day_cos'] = np.cos(2 * np.pi * future['day_of_week'] / 7)
future['is_weekend'] = future['day_of_week'].isin([5,6]).astype(int)

# 2. Populate the Lag Features for the future period
# Propagating historical lags and filling the recursive steps with the mean.
mean_y = df_prepared['y'].mean()
future['lag_24h'] = future['ds'].apply(lambda x: df_prepared[df_prepared['ds'] == (x - pd.Timedelta(hours=24))]['y'].values[0] if (x - pd.Timedelta(hours=24)) in df_prepared['ds'].values else np.nan)
future['lag_168h'] = future['ds'].apply(lambda x: df_prepared[df_prepared['ds'] == (x - pd.Timedelta(hours=168))]['y'].values[0] if (x - pd.Timedelta(hours=168)) in df_prepared['ds'].values else np.nan)

future['lag_24h'] = future['lag_24h'].fillna(mean_y)
future['lag_168h'] = future['lag_168h'].fillna(mean_y)

# Filter the future DataFrame to only include the *new* 30 days
start_of_forecast = df_prepared['ds'].max() + pd.Timedelta(hours=1)
future_forecast = future[future['ds'] >= start_of_forecast].reset_index(drop=True)


# 3. Generate the 30-Day Forecast
forecast_30d = model.predict(future_forecast)

/Users/sanjaym/Desktop/projects/simple_ml_predictor/.venv/lib/python3.13/site-packages/prophet/forecaster.py:1872: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(


In [8]:
# 1. Define Risk: Risk increases with predicted impact (yhat) and with uncertainty (yhat_upper - yhat_lower)
forecast_30d['Uncertainty_Band'] = forecast_30d['yhat_upper'] - forecast_30d['yhat_lower']

# Calculate a Risk Score: High predicted volume + High uncertainty = High Risk
# Risk Score = (Predicted Impact) + (Uncertainty Band)
forecast_30d['Risk_Score'] = forecast_30d['yhat'] + forecast_30d['Uncertainty_Band']

# 2. Extract key time components for analysis
forecast_30d['Hour'] = forecast_30d['ds'].dt.hour
forecast_30d['Date'] = forecast_30d['ds'].dt.strftime('%Y-%m-%d')
forecast_30d['Day_of_Week'] = forecast_30d['ds'].dt.day_name()
forecast_30d['Day_of_Month'] = forecast_30d['ds'].dt.day

# 3. Find Global Minimum and Maximum Risk Hours
min_risk_hour = forecast_30d.sort_values('Risk_Score', ascending=True).iloc[0]
max_risk_hour = forecast_30d.sort_values('Risk_Score', ascending=False).iloc[0]

# 4. Find Top 5 Deployment Windows (Lowest Risk)
recommendations = forecast_30d.sort_values('Risk_Score', ascending=True).head(5)
recommendations = recommendations[['ds', 'Day_of_Week', 'Hour', 'yhat', 'Risk_Score']].copy()
recommendations.columns = ['Date_Time', 'Day_of_Week', 'Hour', 'Predicted_Impact (Norm)', 'Risk_Score']

# 5. Analyze Risk by Hour of Day
risk_by_hour = forecast_30d.groupby('Hour')['Risk_Score'].mean().sort_values(ascending=False).reset_index()

# 6. Analyze Risk by Date of Month and Day of Week
risk_by_month_day = forecast_30d.groupby('Day_of_Month')['Risk_Score'].mean().sort_values(ascending=False).reset_index()
risk_by_day_of_week = forecast_30d.groupby('Day_of_Week')['Risk_Score'].mean().sort_values(ascending=False).reset_index()

# 7. Analyze Risk by Hour and Day Combined
risk_by_hour_day = forecast_30d.groupby(['Day_of_Week', 'Hour'])['Risk_Score'].mean().sort_values(ascending=False).reset_index()
risk_by_hour_day['Rank'] = risk_by_hour_day['Risk_Score'].rank(method='dense', ascending=False).astype(int)


print("\n\n--- 30-DAY DEPLOYMENT RISK ANALYSIS & RECOMMENDATIONS ---")

print("\n[A] GLOBAL MINIMUM RISK DEPLOYMENT WINDOW (The Absolute Safest Hour)")
print(f"Date/Time: {min_risk_hour['ds']}")
print(f"Day of Week: {min_risk_hour['Day_of_Week']}")
print(f"Predicted Impact (Norm): {min_risk_hour['yhat']:,.4f}")
print(f"Minimum Risk Score: {min_risk_hour['Risk_Score']:,.4f}")
print("This is the hour with the lowest combination of predicted traffic and uncertainty in the entire month.")


print("\n[B] GLOBAL MAXIMUM RISK DEPLOYMENT WINDOW (The Absolute Riskiest Hour)")
print(f"Date/Time: {max_risk_hour['ds']}")
print(f"Day of Week: {max_risk_hour['Day_of_Week']}")
print(f"Predicted Impact (Norm): {max_risk_hour['yhat']:,.4f}")
print(f"Maximum Risk Score: {max_risk_hour['Risk_Score']:,.4f}")
print("Deploying during this hour carries the highest combined traffic and uncertainty risk.")


print("\n[C] TOP 5 LOW-RISK DEPLOYMENT WINDOWS (Next Best Suggestions)")
print("Based on the 60:40 weighted impact score (normalized 0 to 1).")
print(recommendations.to_markdown(index=False, floatfmt='.4f'))

print("\n[D] RISK ANALYSIS BY DATE OF MONTH (Where 1 is the riskiest day - Mean Risk)")
# Add rank for easy interpretation
risk_by_month_day['Rank'] = risk_by_month_day['Risk_Score'].rank(method='dense', ascending=False).astype(int)
print(risk_by_month_day[['Rank', 'Day_of_Month', 'Risk_Score']].head(5).to_markdown(index=False, floatfmt='.4f'))

print("\n[E] RISK ANALYSIS BY DAY OF WEEK (Where 1 is the riskiest day - Mean Risk)")
risk_by_day_of_week['Rank'] = risk_by_day_of_week['Risk_Score'].rank(method='dense', ascending=False).astype(int)
# Reorder days for better readability
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
risk_by_day_of_week['Day_of_Week'] = pd.Categorical(risk_by_day_of_week['Day_of_Week'], categories=day_order, ordered=True)
risk_by_day_of_week = risk_by_day_of_week.sort_values('Day_of_Week')
print(risk_by_day_of_week[['Rank', 'Day_of_Week', 'Risk_Score']].to_markdown(index=False, floatfmt='.4f'))

print("\n[F] RISK ANALYSIS BY HOUR OF DAY (Where 1 is the riskiest hour - Mean Risk)")
risk_by_hour['Rank'] = risk_by_hour['Risk_Score'].rank(method='dense', ascending=False).astype(int)
print(risk_by_hour[['Rank', 'Hour', 'Risk_Score']].to_markdown(index=False, floatfmt='.4f'))

print("\n[G] RISK ANALYSIS BY HOUR AND DAY COMBINED (Where 1 is the riskiest window - Mean Risk)")
print("This table ranks all 168 weekly hourly windows (Day + Hour) by their average risk score.")
risk_by_hour_day_safe = risk_by_hour_day.sort_values('Risk_Score', ascending=True).reset_index(drop=True)
risk_by_hour_day_risky = risk_by_hour_day.sort_values('Risk_Score', ascending=False).reset_index(drop=True)

# Reorder days for display
day_order_display = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
risk_by_hour_day_risky['Day_of_Week'] = pd.Categorical(risk_by_hour_day_risky['Day_of_Week'], categories=day_order_display, ordered=True)
risk_by_hour_day_safe['Day_of_Week'] = pd.Categorical(risk_by_hour_day_safe['Day_of_Week'], categories=day_order_display, ordered=True)


print("\nTop 10 Riskiest Windows:")
print(risk_by_hour_day_risky.head(10)[['Rank', 'Day_of_Week', 'Hour', 'Risk_Score']].to_markdown(index=False, floatfmt='.4f'))

print("\nTop 10 Safest Windows:")
print(risk_by_hour_day_safe.head(10)[['Rank', 'Day_of_Week', 'Hour', 'Risk_Score']].to_markdown(index=False, floatfmt='.4f'))




--- 30-DAY DEPLOYMENT RISK ANALYSIS & RECOMMENDATIONS ---

[A] GLOBAL MINIMUM RISK DEPLOYMENT WINDOW (The Absolute Safest Hour)
Date/Time: 2024-02-01 01:00:00
Day of Week: Thursday
Predicted Impact (Norm): 0.5157
Minimum Risk Score: 0.8870
This is the hour with the lowest combination of predicted traffic and uncertainty in the entire month.

[B] GLOBAL MAXIMUM RISK DEPLOYMENT WINDOW (The Absolute Riskiest Hour)
Date/Time: 2024-02-01 17:00:00
Day of Week: Thursday
Predicted Impact (Norm): 1.1430
Maximum Risk Score: 1.5366
Deploying during this hour carries the highest combined traffic and uncertainty risk.

[C] TOP 5 LOW-RISK DEPLOYMENT WINDOWS (Next Best Suggestions)
Based on the 60:40 weighted impact score (normalized 0 to 1).
| Date_Time           | Day_of_Week   |   Hour |   Predicted_Impact (Norm) |   Risk_Score |
|:--------------------|:--------------|-------:|--------------------------:|-------------:|
| 2024-02-01 01:00:00 | Thursday      |      1 |                    0.5157 |